# Homework 2

# Set up

## Installing packages

In [1]:
!pip install requests PyPDF2 gdown
!pip install 'markitdown[pdf]'
!pip install langchain_mcp_adapters langchain_google_genai langchain-openai

## Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `VERTEX_API_KEY`.


1.   Look for the key icon on the left panel of your colab.
2.   Under `Name`, create `VERTEX_API_KEY`.
3. Copy your key to `Value`.

If you cannot use VERTEX_API_KEY, you can use deepseek models via `DEEPSEEK_API_KEY`. It does not affect your score.



In [2]:
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
# DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')

# Download sample CVs

## Downloading sample_cv.pdf
The codes below download the sample CV


In [4]:
import os
import gdown

folder_id = "1adYKq7gSSczFP3iikfA8Er-HSZP6VM7D"
folder_url = f"https://drive.google.com/drive/folders/{folder_id}"

output_dir = "downloaded_cvs"
os.makedirs(output_dir, exist_ok=True)

gdown.download_folder(
    url=folder_url,
    output=output_dir,
    quiet=False,
    use_cookies=False
)

Retrieving folder contents


Processing file 1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp CV_1.pdf
Processing file 16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs CV_2.pdf
Processing file 15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr CV_3.pdf
Processing file 1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk CV_4.pdf
Processing file 1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C CV_5.pdf


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp
To: /content/downloaded_cvs/CV_1.pdf
100%|██████████| 147k/147k [00:00<00:00, 35.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs
To: /content/downloaded_cvs/CV_2.pdf
100%|██████████| 75.1k/75.1k [00:00<00:00, 36.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr
To: /content/downloaded_cvs/CV_3.pdf
100%|██████████| 72.0k/72.0k [00:00<00:00, 45.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk
To: /content/downloaded_cvs/CV_4.pdf
100%|██████████| 73.3k/73.3k [00:00<00:00, 28.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C
To: /content/downloaded_cvs/CV_5.pdf
100%|██████████| 97.9k/97.9k [00:00<00:00, 30.5MB/s]
Download complete

['downloaded_cvs/CV_1.pdf',
 'downloaded_cvs/CV_2.pdf',
 'downloaded_cvs/CV_3.pdf',
 'downloaded_cvs/CV_4.pdf',
 'downloaded_cvs/CV_5.pdf']

In [5]:
# =====================================================
#  Load and display all CV PDFs in order
# =====================================================
import os
from markitdown import MarkItDown

cv_dir = "downloaded_cvs"

# Initialize MarkItDown
md = MarkItDown(enable_plugins=False)

# Collect and sort PDFs numerically
pdf_files = sorted(
    [f for f in os.listdir(cv_dir) if f.lower().endswith(".pdf")],
    key=lambda x: int("".join(filter(str.isdigit, x)))  # CV_1.pdf → 1
)

all_cvs = []

for pdf_name in pdf_files:
    pdf_path = os.path.join(cv_dir, pdf_name)
    result = md.convert(pdf_path)

    all_cvs.append({
        "file": pdf_name,
        "text": result.text_content
    })

    print("=" * 80)
    print(f"📄 {pdf_name}")
    print("=" * 80)
    print(result.text_content)
    print("\n\n")


📄 CV_1.pdf
|     |     |     |     | John         |           | Smith        |                   |     |     |
| --- | --- | --- | --- | ------------ | --------- | ------------ | ----------------- | --- | --- |
|     |     |     |     | Marketing    |           | Professional |                   |     |     |
|     |     |     |     | + Singapore, | Singapore |              | (cid:209) Kowloon |     |     |
Experience
|                |                  |     |          |                     |              |            |     | 2020 – | Present |
| -------------- | ---------------- | --- | -------- | ------------------- | ------------ | ---------- | --- | ------ | ------- |
| Engineer,      | ByteDance        |     |          |                     |              |            |     |        |         |
| • Worked       | in a fast-paced, |     | global   | technology          | environment. |            |     |        |         |
| • Collaborated | across           |     | teams to | sup

# Connect to our MCP server

Documentation about MCP: https://modelcontextprotocol.io/docs/getting-started/intro.

Using MCP servers in Langchain https://docs.langchain.com/oss/python/langchain/mcp.

## Check which tools that the MCP server provide

In [6]:
import asyncio
import json
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
for tool in mcp_tools:
    print(tool.name)
    print(tool.description)
    print(tool.args)
    print("\n\n------------------------------------------------------\n\n")

search_facebook_users
Search for Facebook users by display name (supports partial and fuzzy matching).

Args:
    q: Search query string (case-insensitive, matches any part of display name)
       Examples: "John", "john smith", "Smith"
    limit: Maximum number of results to return (default: 20, max: 20)
    fuzzy: Enable fuzzy matching if exact search returns no results (default: True)

Returns:
    List of user dictionaries, each containing:
    - id (int): Unique Facebook user ID for use with get_facebook_profile()
    - display_name (str): User's Facebook display name (may differ from legal name)
    - city (str): Current city of residence
    - country (str): Country of residence
    - match_type (str): "exact" or "fuzzy" (indicates search method used)
    
    Returns empty list [] if no matches found.

Example:
    search_facebook_users("Alex Chan", limit=5)
    → [{"id": 123, "display_name": "Alex Chan", "city": "Hong Kong", "country": "Hong Kong", "match_type": "exact"}]
    

## A simple agent using tools from the MCP server


In [7]:
import os
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mcp_adapters.client import MultiServerMCPClient

# ---------------------------
# 1. Define a local tool
# ---------------------------
@tool
def say_hello(name: str) -> str:
    """Say hello to a person by name."""
    return f"Hello, {name}! 👋"

# ---------------------------
# 2. Load MCP tools + merge
# ---------------------------
client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
tools = mcp_tools + [say_hello]

# ---------------------------
# 3. Initialize Gemini (tool-enabled) or deepseek
# ---------------------------
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GEMINI_VERTEX_API_KEY,
    vertexai=True,
    temperature=0
)

# from langchain_openai import ChatOpenAI
# DEEPSEEK_API_KEY = userdata.get("DEEPSEEK_API_KEY")
# llm = ChatOpenAI(
#     model="deepseek-chat",          # or "deepseek-reasoner"
#     api_key=DEEPSEEK_API_KEY,
#     base_url="https://api.deepseek.com/v1",
#     temperature=0,
# )

llm_with_tools = llm.bind_tools(tools)

# ---------------------------
# 4. Single-step invocation
# ---------------------------
query = "Say hello to Bao using tool, then search for someone named Alice on Facebook."

response = llm_with_tools.invoke([
    HumanMessage(content=query)
])

print(response)

content='' additional_kwargs={'function_call': {'name': 'search_facebook_users', 'arguments': '{"q": "Alice"}'}, '__gemini_function_call_thought_signatures__': {'45d9e5f7-9802-436c-9005-abc168ab2dff': 'CtkDAY89a1+KSA5chpEZOqe4teyBtnWks4xmlRXJtIv5fZ7oqcw88cvZlLQOZ9IidGZdm9H2pfg4KOuUo+XceGMISe+Kickc1wiFUV1zbabqUfpD7W8BDzXd4kzkzFA5C1Nd7ewwInrEgguJvz6MZ8PbQTFbNRq8nTQ/H6aBlaAT/Ajz+xBhxH4Dk1tUylzLMW7d4ouR2jYPNjvRu71ciV6v0nwfQul/FVhvHe9C8L8Tw1bNSQJ9vmWYxBevEwQKj4MvHb9s1SPXRwhjUVdLQ/eVBcnqM5HkF/zDP65kdaaO3TCl8mYcKROOGX946tG3OxZbKaToH91li12VSJF1l0z9ELqgtZTQsbFmRh0bfv3gTitCy3uftwM+rZWTolmAYabc3/vSwqQhjj0TDo/OrXh2GBP5fYMuSaGaJoauIbRchLaJUOoJSW7eqbVITL9P3NJfFQ9eox/8HZ9oPPc4rmH6piqW/N95PsmOpZLAuuqwMfxlv6xM8pDYOBJnbN4nqikD8rWqn01ovTiT3lvCYvsrgT0fr497YMm4eWSRoHCLX5gDaLt1mhV1UKmRRLjSgOl+DBlepnTMljc/klBgFmaijGVvPNF8WNPbX6JDr6OXMojqY7rvmG9z6Io='}} response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019c9e8e-42a

In [11]:
import re
import json
from langchain_core.messages import HumanMessage, ToolMessage, SystemMessage
from langchain_core.output_parsers import JsonOutputParser

async def verify_cv_process(cv_item):
    filename = cv_item.get('file', 'Unknown')
    raw_text = cv_item.get('text', '')

    # clean data
    clean_text = re.sub(r'[|\\-]{2,}', ' ', raw_text)
    clean_text = re.sub(r'\(cid:\d+\)', ' ', clean_text)
    clean_text = re.sub(r'\s+', ' ', clean_text).strip()

    if len(clean_text) < 20:
        return f"{filename}: the content is insufficient"

    print(f"the text is being processed: {filename} (length: {len(clean_text)})")

    # --- A：fetch information ---
    extract_prompt = f"Please extract (name, current_company, current_title, education, experience) from this messy PDF text into JSON:\n\n{clean_text}"

    try:
        messages = [HumanMessage(content=extract_prompt)]
        resp = llm.invoke(messages)

        if not resp or not resp.content:
            return "❌ LLM the content is none"

        cv_json = JsonOutputParser().parse(resp.content)
        print(f"✅ success: {cv_json.get('name')}")
    except Exception as e:
        return f"❌ error: {str(e)}"

    # --- B：tool verification ---
    agent_instr = f"Identify discrepancies for {cv_json.get('name')}. Data: {json.dumps(cv_json)}. Check for future dates (2027), job overlaps, or skill-role mismatch (e.g. Marketing degree but Engineer job)."
    agent_msgs = [HumanMessage(content=agent_instr)]

    for _ in range(5):
        response = llm_with_tools.invoke(agent_msgs)
        if not response: break
        agent_msgs.append(response)

        if not response.tool_calls: break

        for tool_call in response.tool_calls:
            selected_tool = next((t for t in tools if t.name == tool_call["name"]), None)
            if selected_tool:
                obs = await selected_tool.ainvoke(tool_call["args"])
                obs_str = str(obs) if obs is not None else "No data found."
                agent_msgs.append(ToolMessage(tool_call_id=tool_call["id"], content=obs_str))

    return agent_msgs[-1].content

In [12]:
# make sure the list of all_cvs have data
if not all_cvs:
    print("❌ error：all_cvs is empty")
else:
    print(f"🚀 deal with {len(all_cvs)} cv...\n")

    all_final_reports = []

    for cv in all_cvs:
        report = await verify_cv_process(cv)
        all_final_reports.append({
            "file": cv['file'],
            "report": report
        })

    print("\n" + "#"*30 + " final result " + "#"*30)
    for res in all_final_reports:
        print(f"\n📄 file: {res['file']}")
        print(f"{res['report']}")
        print("-" * 60)

🚀 deal with 5 cv...

the text is being processed: CV_1.pdf (length: 733)
✅ success: John Smith
the text is being processed: CV_2.pdf (length: 943)
✅ success: Minh Pham
the text is being processed: CV_3.pdf (length: 723)
✅ success: Wei Zhang
the text is being processed: CV_4.pdf (length: 1231)
✅ success: Rahul Sharma
the text is being processed: CV_5.pdf (length: 1299)
✅ success: Rahul Sharma

############################## final result ##############################

📄 file: CV_1.pdf
[{'type': 'text', 'text': 'Based on the data provided for John Smith, I\'ve identified the following potential discrepancy:\n\n*   **Skill-role mismatch**: John Smith has a "Bachelor of Science (BSc) in Marketing" from McGill University, but his current role is "Engineer" at ByteDance. This suggests a potential mismatch between his educational background and his current profession.', 'extras': {'signature': 'CvQJAY89a19ORqb36V2ZIpiQV5GkVbsfX4jETZSSGrcs/CwoJMtt8oOniZ1tS7r97Zzmc+xm9cCGEzfDE5KoZBND34tsELApGvl

In [13]:
# auto score system
# =====================================================

final_scores = []

for r in all_final_reports:
    report_text = r['report']

    scoring_prompt = f"""
    Role: Senior Risk Assessor (Verification Task)
    Task: Provide a reliability score [0.0 to 1.0] based on the report.

    SCORING RULES (Strict Compliance):
    1. VALID CV (Score 0.7 - 1.0):
       - Even if the LinkedIn search result doesn't perfectly match (e.g., CV says PwC, LinkedIn says Tencent), the CV is considered VALID.
       - A2 Rule: Do NOT reject a CV just because an exact social profile match is not found.

    2. INVALID CV (Score 0.0 - 0.4):
       - Only give a low score if there are INJECTED logical inconsistencies:
         * Future dates (e.g., jobs ending in 2027).
         * Impossible time overlaps (simultaneous full-time jobs).
         * Glaring internal contradictions in the text.

    Evaluate this report: "{report_text}"

    Output ONLY the numerical score (e.g., 0.85).
    """

    score_resp = llm.invoke([HumanMessage(content=scoring_prompt)])
    try:
        score = float(score_resp.content.strip())
    except:
        score = 0.5

    final_scores.append(score)

print(f"the score is: {final_scores}")

the score is: [0.9, 0.95, 0.7, 0.1, 0.1]


In [14]:
# This block provides you some tests to get faminilar with our MCP server

# # Test 1: Search Facebook users (exact match)
# await tools[0].ainvoke({'q': "Alex Chan", 'limit': 5})

# # Test 2: Search Facebook users (fuzzy match with typo)
# await tools[0].ainvoke({'q': "Alx Chn", 'limit': 5, 'fuzzy': True})

# # Test 3: Get Facebook profile
# await tools[1].ainvoke({'user_id': 123})

# # Test 4: Get Facebook mutual friends
# await tools[2].ainvoke({'user_id_1': 123, 'user_id_2': 456})

# # Test 5: Search LinkedIn people (exact match)
# await tools[3].ainvoke({'q': "Python", 'location': "Hong Kong", 'limit': 5})

# # Test 6: Search LinkedIn people (fuzzy match with typo)
# await tools[3].ainvoke({'q': "Python", 'location': "Hong Kong", 'limit': 5, 'fuzzy': True})

# # Test 7: Get LinkedIn profile
# await tools[4].ainvoke({'person_id': 456})

# Test 8: Get LinkedIn interactions
await tools[5].ainvoke({'person_id': 456})

[{'type': 'text',
  'text': '{"profile_id":456,"post_count":4,"total_likes":5,"liked_by":[4390,3622,7500,4269,8464],"engagement_score":1.25}',
  'id': 'lc_ddf12768-24b9-4fd8-93c4-d90dc848b315'}]

# Evaluation code

In the test phase, you will be given 5 CV files with fixed names:

    CV_1.pdf, CV_2.pdf, CV_3.pdf, CV_4.pdf, CV_5.pdf

Your system must process these CVs and output a list of 5 scores,
one score per CV, in the same order:

    scores = [s1, s2, s3, s4, s5]

Each score must be a float in the range [0, 1], representing the
reliability or confidence that the CV is valid (or meets the task criteria).

The ground-truth labels are binary:

    groundtruth = [0 or 1, ..., 0 or 1]

Each CV is evaluated independently using a threshold of 0.5:

- If score > 0.5 and groundtruth == 1 → Full credit
- If score ≤ 0.5 and groundtruth == 0 → Full credit
- Otherwise → No credit

In other words, 0.5 is the decision threshold.

- Each CV contributes equally.
- Final score = (number of correct decisions) / 5


In [15]:
# =====================================================
#  Evaluation code
# =====================================================

def evaluate(scores, groundtruth, threshold=0.5):
    """
    scores: list of floats in [0, 1], length = 5
    groundtruth: list of ints (0 or 1), length = 5
    """
    assert len(scores) == 5
    assert len(groundtruth) == 5

    correct = 0
    decisions = []

    for s, gt in zip(scores, groundtruth):
        pred = 1 if s > threshold else 0
        decisions.append(pred)
        if pred == gt:
            correct += 1

    final_score = correct / len(scores)

    return {
        "decisions": decisions,
        "correct": correct,
        "total": len(scores),
        "final_score": final_score
    }


In [16]:
scores = [0.9, 0.95, 0.7, 0.1, 0.1] # Your code should generate this list [0.2, 0.3, 0.4, 0.5, 0.6]
groundtruth = [1, 1, 1, 0, 0] # Do not modify

result = evaluate(scores, groundtruth)
print(result)


{'decisions': [1, 1, 1, 0, 0], 'correct': 5, 'total': 5, 'final_score': 1.0}
